In [56]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import copy
import time 
import matplotlib.pyplot as plt
import pickle # Lokales Speichern von Objekten
import keyboard
from collections import Counter

from GNM_Toolbox.tools.tools import *
from GNM_Toolbox.gnm import *
from GNM_Toolbox.data.dataloader import *

dataset = load_dataset('Citeseer')

Found dataset on harddrive.


# Some Helping Functions

In [57]:
# Gegeben sei eine target_list (a_0, a_1, a_2, ...)
# und eine out_list ((b_0, x), (b_1, x), (b_2, x), (b_3, x), ...)
# Gesucht wird eine Liste l von Indizes, sodass für i < len(target_list): abs(target_list[i] - out_list[l[i]][0]) minimal ist
def find_each_nearest(target_list, out_list):
    # Each list is expected to be sorted
    i, j = 0, 0
    result = list()
    while True:
        diff_0 = abs(target_list[i] - out_list[j][0])
        diff_1 = abs(target_list[i] - out_list[j+1][0])
        
        if diff_0 >= diff_1:
            j += 1
        elif diff_0 < diff_1:
            result.append(j)
            i += 1
        if i >= len(target_list):
            return result
        if j+1 >= len(out_list):
            while i < len(target_list):
                result.append(j)
                i += 1
            return result
            
def get_best_values_indices(targets, lambdas):
    lambdas.sort(key = lambda x: x[0])
    return find_each_nearest(targets, lambdas)

def h(x):
    a0, a1, a2, a3 = 13, 4, 15, 15
    return torch.exp(torch.sum(x, dim=1)/a0 - a1) - ((torch.sum(x, dim=1) - a2 ) / a3)
    
def pi(X, y):
    a0, a1, a2 = -torch.log(torch.tensor(4.)), 1, -1
    return torch.sigmoid(a0 + a1 * h(X) + a2 * y)

def create_mask_from_pi(data, pi):
    p = pi(data.x, data.y)
    mask = torch.tensor((np.random.binomial(size = p.shape[0], n = 1, p = p) == 1))        
    return mask.bool()

def split_known_mask_into_val_and_train_mask(known, ratio=0.8):
    val_mask = torch.zeros_like(known) == 1
    train_mask = torch.zeros_like(known) == 1
    for i in range(len(known)):
        if known[i] == True:
            if np.random.binomial(1, ratio) == 1:
                train_mask[i] = True
            else:
                val_mask[i] = True
    return val_mask, train_mask

def calculate_lambda(train_mask, y):
    a = 0 # Anzahl an Klasse 0
    b = 0 # Anzahl an Klasse 1
    for yy in y[train_mask]:
        if yy == 0:
            a += 1
        elif yy == 1:
            b += 1
    return b/a
        
def insert_into_list(l, item, t):
    # l list, i item to insert, target
    def diff(a, b):
        return abs(a-b)
    N = len(l)
    if N == 0:
        l.insert(0, item)
        return
    d = diff(t, item[0])
    d_0 = diff(t, l[0][0])
    if d <= d_0:
        l.insert(0, item)
        return
    for i in range(N-1):
        d_0 = diff(t, l[i][0])
        d_1 = diff(t, l[i+1][0])
        if d_0 <= d and d <= d_1:
            l.insert(i+1, item)
            return
    l.append(item)

# Setup Informations:

Citeseer Datasets consists of 6 classes.

Node distribution is as follows 249, 590, 668, 701, 596, 523.

Therefore we set new class 0 to be class 3. New class 1 is going to be the rest.

We have 3327 nodes with 3703 features each. 

Citation:
@inproceedings{nr,
     title={The Network Data Repository with Interactive Graph Analytics and Visualization},
     author={Ryan A. Rossi and Nesreen K. Ahmed},
     booktitle={AAAI},
     url={http://networkrepository.com},
     year={2015}
}

# Data Setup

In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
y_tmp = data.y == 3
data.y = torch.ones_like(data.y)
data.y[y_tmp] = 0
data.num_classes = 2
print()

In [47]:
lambdas = list()

for i in range(1000):
    mask = create_mask_from_pi(data, pi)
    var_lambda = calculate_lambda(mask, data.y)
    lambdas.append(var_lambda)
    print('{:.3f}'.format(sum(mask)/(len(mask)*1.0)), end='\r')
    time.sleep(1)

plt.hist(lambdas, bins=40);

KeyboardInterrupt: 

# Find Fitting Train and Val Masks

In [48]:
# Find masks such that lambda has specific values
targets = [1, 1.5, 2]
masks = dict()
fitting_masks = dict()
targets_done = set()
worst_allowed_diff = 0.01
worst_diff = 1
num_masks = 40 # num_masks_per_lambda

for t in targets:
    masks[t] = list()
    fitting_masks[t] = 0
    
pickle_write('masks_citeseer_small.pkl', (masks, fitting_masks, targets_done))

def print_fitting(fitting_masks, i):
    k = fitting_masks.keys()
    string = ''
    for kk in k:
        if fitting_masks[kk] <= 40:
            string += ' {}'.format(fitting_masks[kk])
        else:
            string += ' 40+'
    print(string + ' at iteration {}'.format(i), end='\r')

In [49]:
masks, fitting_masks, targets_done = pickle_read('masks_citeseer_small.pkl')
i = 0
t_start = time.time()
while worst_diff > worst_allowed_diff and i < 40000 and len(set(targets)-targets_done) > 0:
    i += 1
    known_mask = create_mask_from_pi(data, pi)
    val_mask, train_mask = split_known_mask_into_val_and_train_mask(known_mask)
    l = calculate_lambda(train_mask, data.y)
    item = (l, train_mask, val_mask)
    
    # Update masks
    for t in set(targets)-targets_done:
        diff = abs(l - t)
        masks_ind = masks[t]
        if len(masks_ind) < num_masks:
            insert_into_list(masks_ind, item, t)
            if diff < worst_allowed_diff:
                fitting_masks[t] += 1
                #print_fitting(fitting_masks, i)
                if fitting_masks[t] > 40:
                    targets_done.add(t)
        elif abs(masks_ind[num_masks-1][0] - t) > diff:
            insert_into_list(masks_ind, item, t)
            masks[t] = masks_ind[0:num_masks]
            if diff < worst_allowed_diff:
                fitting_masks[t] += 1
                print_fitting(fitting_masks, i)
                if fitting_masks[t] > 40:
                    targets_done.add(t)
    
    # Update worst_diff
    worst_diff_old = worst_diff
    worst_diff = 0
    for t in set(targets)-targets_done:
        length = len(masks[t])
        diff = abs(t - masks[t][length-1][0])
        if diff > worst_diff:
            worst_diff = diff
            
# Safe masks
pickle_write('masks_citeseer_small.pkl', (masks, fitting_masks, targets_done))

## Analysis if Loss Function $L_1$ ist actually better than NLL

In [59]:
# Evaluate actuall pi
pi_true = pi(data.x, data.y)

# Load masks
all_masks,_,_ = pickle_read('masks_citeseer_small.pkl')
subset = [1, 1.5, 2]
choosen_masks = {k: all_masks[k] for k in subset}

# Drei Masken: 10, 15, 20
# Acht +1 Noiselevel: 0, 0.0025, 0.005, 0.0075, 0.01, 0.0125, 0.015, 0.0175, 0.02

In [60]:
# Eval 'Perfect' Loss, 'Perfect' Loss with noise and NLL
IT_per_mask = 4
NB_masks = len(choosen_masks[1.0])
M = len(choosen_masks)
t_0 = time.time()
noise_levels = [0, 0.0025, 0.005, 0.0075, 0.01, 0.0125, 0.015, 0.0175, 0.02]
all_models = dict()

# Iteriere über Masken
for i, l in enumerate(choosen_masks):
    sms_models = list() # SM Standard
    smn_models = [list() for _ in range(9)] # SM Advanced with noises
    
    for j, mask_tupel in enumerate(choosen_masks[l]):
        _, train_mask, val_mask = mask_tupel

        # Trainiere jeweils N Modelle
        for k in range(IT_per_mask):
            print_status(i * NB_masks * IT_per_mask + j * IT_per_mask + k, M * NB_masks * IT_per_mask, t_0)
            noise = torch.randn_like(pi_true)
            for m, noise_level in enumerate(noise_levels):
                pi = pi_true + noise_level * noise
                smn_models[m].append((*train_one_net(data, 
                                                  train_mask, 
                                                  val_mask,
                                                  loss_function=weighted_categorial_crossentropy_loss(pi[train_mask], reduction='mean'),
                                                  val_loss_function=weighted_categorial_crossentropy_loss(pi[val_mask], reduction='mean'))[1:], j))
            sms_models.append((*train_one_net(data, train_mask, val_mask)[1:], j))
        all_models[l] = (sms_models, *smn_models)
        pickle_write('l1_analysis-citeseer_small.pkl', all_models)
    all_models[l] = (sms_models, *smn_models)
    
pickle_write('l1_analysis-citeseer_small.pkl', all_models)